In [2]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge, RidgeCV
import gc
from catboost import CatBoostRegressor
import seaborn as sns

In [3]:
train = pd.read_csv('../input/train.csv', nrows = 1000000, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

In [4]:
train.head(20)

,acoustic_data,time_to_failure
0,12,1.4690999832
1,6,1.4690999821
2,8,1.4690999810
3,5,1.4690999799
4,8,1.4690999788
5,8,1.4690999777
6,9,1.4690999766
7,7,1.4690999755
8,-5,1.4690999744
9,3,1.4690999733


In [5]:
train.describe()

,acoustic_data,time_to_failure
count,1000000.000000000000000,1000000.000000000000000
mean,4.876248000000000,1.339787547164501
std,6.380823156725169,0.074985946080878
min,-199.000000000000000,1.209499350700000
25%,2.000000000000000,1.274399508825000
50%,5.000000000000000,1.339299666950000
75%,7.000000000000000,1.404199825075000
max,197.000000000000000,1.469099983200000


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
acoustic_data      1000000 non-null int16
time_to_failure    1000000 non-null float64
dtypes: float64(1), int16(1)
memory usage: 9.5 MB


In [8]:
train.shape

(1000000, 2)

In [10]:
train.columns

Index(['acoustic_data', 'time_to_failure'], dtype='object')

In [12]:
train["sma"] = train["acoustic_data"].rolling(window = 5).mean()
train.head(10)

,acoustic_data,time_to_failure,sma
0,12,1.4690999832,NaN
1,6,1.4690999821,NaN
2,8,1.4690999810,NaN
3,5,1.4690999799,NaN
4,8,1.4690999788,7.8
5,8,1.4690999777,7.0
6,9,1.4690999766,7.6
7,7,1.4690999755,7.4
8,-5,1.4690999744,5.4
9,3,1.4690999733,4.4


In [15]:
train["ewma"] = pd.Series.ewm(train['acoustic_data'], span=5).mean()
train.head(10)

,acoustic_data,time_to_failure,sma,ewma
0,12,1.4690999832,NaN,12.000000000000000
1,6,1.4690999821,NaN,8.400000000000000
2,8,1.4690999810,NaN,8.210526315789473
3,5,1.4690999799,NaN,6.876923076923076
4,8,1.4690999788,7.8,7.308056872037914
5,8,1.4690999777,7.0,7.560902255639098
6,9,1.4690999766,7.6,8.070422535211268
7,7,1.4690999755,7.4,7.699127676447264
8,-5,1.4690999744,5.4,3.353033227270357
9,3,1.4690999733,4.4,3.233278759155537
